# CONFLUENCE Tutorial - 11: CARAVAN Large Sample Study (Global Multi-Basin Streamflow Analysis)

## Introduction

This tutorial demonstrates systematic streamflow modeling across multiple watersheds using the CARAVAN dataset (Kratzert et al., 2023). Unlike previous tutorials focused on regional analysis, this study addresses global-scale watershed streamflow prediction across diverse international environmental conditions and hydrological regimes.

### CARAVAN Dataset

The CARAVAN dataset provides standardized data for over 9,000 watersheds across multiple continents, representing the most comprehensive global collection of catchment-scale hydrological data. The dataset includes harmonized meteorological forcing, quality-controlled daily discharge observations, and comprehensive catchment attributes spanning North America, Europe, Australia, Brazil, and Chile. Watersheds range from small headwater basins to large river systems and encompass the full spectrum of global climate zones while maintaining focus on basins with reliable observational records.

### Global Streamflow Modeling Challenges

Streamflow represents the integrated watershed response to precipitation, evapotranspiration, snowmelt, groundwater interactions, and routing processes. Global multi-basin analysis presents unique challenges including extreme spatial heterogeneity across continents, diverse climate regimes from arctic to tropical conditions, varying physiographic controls across different geological provinces, multiple temporal dynamics spanning sub-daily to multi-decadal scales, and scale interactions between local processes and regional climate patterns.

### Research Objectives

This tutorial addresses fundamental questions about universal hydrological principles across global environmental gradients, model performance variations across different climate zones and physiographic regions, parameter transferability between watersheds across continents, streamflow sensitivity to diverse climate forcing patterns, and identification of globally consistent versus regionally specific hydrological behaviors. The analysis employs multiple performance metrics including Nash-Sutcliffe efficiency, Kling-Gupta efficiency, bias assessment, and flow signature analysis.

### Methodological Framework

The approach involves strategic site selection across global environmental gradients, standardized model configuration adaptable to diverse basin characteristics worldwide, automated batch processing execution across multiple continental regions, and systematic performance evaluation using internationally comparable metrics. Sites are selected to represent global climate diversity from polar to tropical zones, physiographic variation across different geological and topographic settings, multiple watershed scales from headwaters to major river basins, and diverse hydrological regimes including snow-dominated, rain-dominated, and mixed systems.

### CONFLUENCE Advantages for Global Analysis

CONFLUENCE provides consistent methodology across diverse global watersheds, automated processing capabilities scalable to thousands of basins, systematic quality control adaptable to different data standards, and comprehensive uncertainty assessment across varying environmental conditions. The framework emphasizes process-based modeling with flexible structure adaptable to different global watershed characteristics, standardized output formats enabling cross-regional comparison, and robust performance evaluation suitable for international hydrological studies.

### Expected Outcomes

This tutorial demonstrates global watershed-scale configuration across multiple continents, streamflow validation through comprehensive observed-simulated comparisons, performance analysis across diverse climate zones and physiographic regions, identification of universal versus region-specific hydrological patterns, and process diagnostics revealing global controls on watershed function. Results contribute to improved understanding of global hydrological controls, enhanced model development for international applications, and applications in global water resources assessment and climate impact evaluation.

## Step 1: Global Multi-Basin Streamflow Experimental Design and Site Selection
Transitioning from regional CAMELS-SPAT analysis to comprehensive global streamflow hydrology simulations, 
this step establishes the foundation for large sample hydrological modeling using the comprehensive CARAVAN dataset. 
We demonstrate how CONFLUENCE's workflow efficiency enables systematic streamflow evaluation across the full 
spectrum of global hydroclimate conditions spanning multiple continents.


In [ ]:

import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import yaml
from datetime import datetime
import seaborn as sns
import warnings
import glob

# Set up plotting style for global watershed visualization
plt.style.use('default')
sns.set_palette("viridis")
%matplotlib inline
confluence_path = Path('../').resolve()

# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/anvil/scratch/x-deythorsson/CONFLUENCE_data')  # Update this path
#CONFLUENCE_DATA_DIR = Path('/path/to/your/CONFLUENCE_data') 

# =============================================================================
# CARAVAN GLOBAL TEMPLATE CONFIGURATION
# =============================================================================

# Load streamflow configuration template or create from base template
streamflow_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'
with open(streamflow_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update for CARAVAN tutorial-specific settings
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'caravan_template',
    'EXPERIMENT_ID': 'run_1',
    'EXPERIMENT_TIME_START': '2000-01-01 01:00',
    'EXPERIMENT_TIME_END': '2020-12-31 23:00',  # 20-year period for global streamflow analysis
}

config_dict.update(config_updates)

# Save CARAVAN configuration template
caravan_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_caravan_template.yaml'
with open(caravan_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"CARAVAN global template configuration saved: {caravan_config_path}")

# =============================================================================
# LOAD AND EXAMINE CARAVAN GLOBAL WATERSHED DATASET
# =============================================================================

print(f"\nLoading CARAVAN Global Watershed Database...")

# Load the CARAVAN watersheds database
try:
    caravan_df = pd.read_csv('caravan-global-metadata.csv')
    print(f"Successfully loaded CARAVAN database: {len(caravan_df)} watersheds available")
except FileNotFoundError:
    print(f"CARAVAN database not found, creating demonstration dataset...")
    
    # Create comprehensive demonstration CARAVAN dataset for tutorial
    np.random.seed(42)
    n_watersheds = 200
    
    # Generate realistic global watershed locations across continents
    # Focus on major hydrological regions with good streamflow data
    regions = [
        # North America
        {'name': 'NA_Pacific_Northwest', 'lat_range': (42, 55), 'lon_range': (-135, -117), 'n': 25, 'continent': 'North America'},
        {'name': 'NA_Rocky_Mountains', 'lat_range': (37, 50), 'lon_range': (-115, -105), 'n': 30, 'continent': 'North America'},
        {'name': 'NA_Great_Lakes', 'lat_range': (42, 49), 'lon_range': (-95, -75), 'n': 20, 'continent': 'North America'},
        {'name': 'NA_Southeastern', 'lat_range': (25, 40), 'lon_range': (-95, -75), 'n': 25, 'continent': 'North America'},
        
        # Europe
        {'name': 'EU_Scandinavia', 'lat_range': (55, 70), 'lon_range': (5, 30), 'n': 20, 'continent': 'Europe'},
        {'name': 'EU_Central_Europe', 'lat_range': (45, 55), 'lon_range': (5, 25), 'n': 25, 'continent': 'Europe'},
        {'name': 'EU_Mediterranean', 'lat_range': (35, 45), 'lon_range': (-5, 25), 'n': 15, 'continent': 'Europe'},
        
        # Australia
        {'name': 'AU_Eastern', 'lat_range': (-40, -25), 'lon_range': (140, 155), 'n': 20, 'continent': 'Australia'},
        {'name': 'AU_Southeastern', 'lat_range': (-40, -30), 'lon_range': (130, 150), 'n': 15, 'continent': 'Australia'},
        
        # South America
        {'name': 'SA_Brazil_Atlantic', 'lat_range': (-25, -5), 'lon_range': (-50, -35), 'n': 10, 'continent': 'South America'},
        {'name': 'SA_Chile_Central', 'lat_range': (-40, -30), 'lon_range': (-75, -70), 'n': 10, 'continent': 'South America'}
    ]
    
    watersheds_data = []
    watershed_id = 1
    
    for region in regions:
        for i in range(region['n']):
            lat = np.random.uniform(region['lat_range'][0], region['lat_range'][1])
            lon = np.random.uniform(region['lon_range'][0], region['lon_range'][1])
            
            # Area based on typical CARAVAN watersheds (log-normal distribution)
            area = np.random.lognormal(np.log(500), 1.5)
            area = np.clip(area, 10, 50000)  # Clip to reasonable global range
            
            # Elevation varies by region and continent
            if 'Rocky_Mountains' in region['name'] or 'Chile' in region['name']:
                elevation = np.random.uniform(1500, 4000)
            elif 'Scandinavia' in region['name']:
                elevation = np.random.uniform(200, 1500)
            elif 'Australia' in region['continent']:
                elevation = np.random.uniform(50, 800)
            elif region['continent'] == 'Europe':
                elevation = np.random.uniform(100, 2000)
            else:
                elevation = np.random.uniform(50, 2500)
            
            # Climate characteristics affecting streamflow - vary by continent and latitude
            # Temperature based on latitude and continent
            if abs(lat) < 20:  # Tropical
                mat_temp = np.random.uniform(20, 28)
                map_precip = np.random.uniform(1000, 3000)
            elif abs(lat) < 40:  # Temperate
                mat_temp = np.random.uniform(8, 20)
                map_precip = np.random.uniform(400, 2000)
            elif abs(lat) < 60:  # Boreal/Cool temperate
                mat_temp = np.random.uniform(-5, 15)
                map_precip = np.random.uniform(300, 1500)
            else:  # Arctic/Subarctic
                mat_temp = np.random.uniform(-15, 5)
                map_precip = np.random.uniform(200, 1000)
            
            # Adjust for regional patterns
            if region['continent'] == 'Australia':
                map_precip *= 0.7  # Generally drier
            elif 'Mediterranean' in region['name']:
                map_precip = np.random.uniform(300, 800)  # Mediterranean climate
            elif 'Pacific_Northwest' in region['name']:
                map_precip = np.random.uniform(1200, 3500)  # Very wet
            
            # Derived characteristics
            pet = max(1, (mat_temp + 5) * 365 * 0.5)  # Simple PET estimate
            aridity = pet / map_precip if map_precip > 0 else 10
            seasonality = np.random.uniform(0.1, 0.9)  # Precipitation seasonality
            
            # Snow fraction based on temperature and elevation
            if mat_temp < -2:
                snow_fraction = np.random.uniform(0.6, 0.95)
            elif mat_temp < 5 and elevation > 1000:
                snow_fraction = np.random.uniform(0.3, 0.8)
            elif mat_temp < 10 and elevation > 2000:
                snow_fraction = np.random.uniform(0.2, 0.6)
            else:
                snow_fraction = np.random.uniform(0.0, 0.3)
            
            # Forest fraction varies by climate and continent
            if region['continent'] == 'Australia':
                forest_frac = np.random.uniform(0.05, 0.5)
            elif 'Mediterranean' in region['name']:
                forest_frac = np.random.uniform(0.1, 0.6)
            elif mat_temp > 15 and map_precip > 1000:  # Tropical/humid
                forest_frac = np.random.uniform(0.6, 0.95)
            elif mat_temp > 5 and map_precip > 600:  # Temperate
                forest_frac = np.random.uniform(0.3, 0.8)
            else:  # Arid/cold
                forest_frac = np.random.uniform(0.05, 0.4)
            
            # Hydro-climatic classification
            if aridity > 3:
                climate_class = 'Arid'
            elif aridity > 1.5:
                climate_class = 'Semi-arid'
            elif aridity > 0.8:
                climate_class = 'Sub-humid'
            else:
                climate_class = 'Humid'
            
            # Scale classification based on area
            if area < 100:
                scale = 'headwater'
            elif area < 1000:
                scale = 'meso'
            elif area < 10000:
                scale = 'macro'
            else:
                scale = 'large'
            
            # Streamflow characteristics
            runoff_coeff = np.random.uniform(0.1, 0.7)
            if climate_class in ['Humid', 'Sub-humid']:
                runoff_coeff = np.random.uniform(0.3, 0.8)
            elif climate_class == 'Semi-arid':
                runoff_coeff = np.random.uniform(0.1, 0.4)
            else:  # Arid
                runoff_coeff = np.random.uniform(0.05, 0.2)
            
            mean_q = area * map_precip * 0.001 * runoff_coeff / 31.536  # Convert to m³/s
            baseflow_index = np.random.uniform(0.1, 0.8)
            
            # Flow regime classification
            if snow_fraction > 0.5:
                flow_regime = 'snow_dominated'
            elif snow_fraction > 0.2:
                flow_regime = 'mixed'
            else:
                flow_regime = 'rain_dominated'
            
            # Create watershed entry
            watershed = {
                'gauge_id': f"CARAVAN_{region['continent'][:2]}_{watershed_id:05d}",
                'gauge_name': f"{region['name']}_Basin_{i+1:03d}",
                'country': region['continent'],
                'continent': region['continent'],
                'gauge_lat': round(lat, 4),
                'gauge_lon': round(lon, 4),
                'area': round(area, 1),
                'elev_mean': round(elevation, 0),
                'p_mean': round(map_precip, 0),  # Mean annual precipitation
                't_mean': round(mat_temp, 1),    # Mean annual temperature
                'pet_mean': round(pet, 0),       # Potential ET
                'aridity': round(aridity, 3),
                'seasonality_p': round(seasonality, 3),
                'frac_snow': round(snow_fraction, 3),
                'forest_frac': round(forest_frac, 3),
                'q_mean': round(mean_q, 2),
                'runoff_ratio': round(runoff_coeff, 3),
                'baseflow_index': round(baseflow_index, 3),
                'climate_class': climate_class,
                'flow_regime': flow_regime,
                'scale': scale,
                'region': region['name'],
                'data_length': np.random.randint(15, 25),  # Years of data
                'data_quality': np.random.choice(['excellent', 'good', 'fair'], p=[0.4, 0.5, 0.1])
            }
            
            # Add CONFLUENCE formatting
            buffer = 0.1
            watershed['BOUNDING_BOX_COORDS'] = f"{lat + buffer}/{lon - buffer}/{lat - buffer}/{lon + buffer}"
            watershed['POUR_POINT_COORDS'] = f"{lat}/{lon}"
            watershed['Watershed_Name'] = watershed['gauge_id'].replace(' ', '_')
            
            watersheds_data.append(watershed)
            watershed_id += 1
    
    caravan_df = pd.DataFrame(watersheds_data)
    
    # Save demonstration dataset
    caravan_df.to_csv('caravan-global-metadata.csv', index=False)
    print(f"Created demonstration CARAVAN dataset: {len(caravan_df)} watersheds")

# Display basic dataset information
print(f"\nGlobal Dataset Overview:")
print(f"  Total watersheds: {len(caravan_df)}")
print(f"  Columns: {len(caravan_df.columns)}")
print(f"  Column names: {', '.join(caravan_df.columns[:8])}...")

# Extract coordinates for analysis
if 'gauge_lat' in caravan_df.columns and 'gauge_lon' in caravan_df.columns:
    caravan_df['latitude'] = caravan_df['gauge_lat']
    caravan_df['longitude'] = caravan_df['gauge_lon']
    caravan_df['drainage_area'] = caravan_df['area']
    
    print(f"Coordinate extraction successful")
    print(f"  Latitude range: {caravan_df['latitude'].min():.1f}° to {caravan_df['latitude'].max():.1f}°")
    print(f"  Longitude range: {caravan_df['longitude'].min():.1f}° to {caravan_df['longitude'].max():.1f}°")
    print(f"  Drainage area range: {caravan_df['drainage_area'].min():.0f} to {caravan_df['drainage_area'].max():.0f} km²")

# =============================================================================
# GLOBAL WATERSHED-SPECIFIC DATASET CHARACTERISTICS ANALYSIS
# =============================================================================

print(f"\nAnalyzing Global Watershed Dataset Characteristics...")

# Continental distribution
if 'continent' in caravan_df.columns:
    continent_counts = caravan_df['continent'].value_counts()
    print(f"  Continental distribution: {len(continent_counts)} continents")
    for continent, count in continent_counts.items():
        print(f"    {continent}: {count} watersheds")

# Area-based watershed scale zones
area_zones = [
    (0, 100, 'Headwater'),
    (100, 1000, 'Meso-scale'),
    (1000, 10000, 'Macro-scale'),
    (10000, 100000, 'Large-scale')
]

caravan_df['area_class'] = 'Unknown'
for min_area, max_area, zone_name in area_zones:
    mask = (caravan_df['drainage_area'] >= min_area) & (caravan_df['drainage_area'] < max_area)
    caravan_df.loc[mask, 'area_class'] = zone_name

area_counts = caravan_df['area_class'].value_counts()
print(f"  Watershed scales: {len(area_counts)}")
print(f"    Most common: {area_counts.index[0]} ({area_counts.iloc[0]} watersheds)")

# Climate-based zones using aridity
if 'aridity' in caravan_df.columns:
    climate_counts = caravan_df['climate_class'].value_counts()
    print(f"  Climate zones: {len(climate_counts)}")
    for climate, count in climate_counts.items():
        print(f"    {climate}: {count} watersheds")

# Flow regime analysis
if 'flow_regime' in caravan_df.columns:
    regime_counts = caravan_df['flow_regime'].value_counts()
    print(f"  Flow regimes: {len(regime_counts)}")
    for regime, count in regime_counts.items():
        print(f"    {regime}: {count} watersheds")

# Climate characteristics
if 'p_mean' in caravan_df.columns:
    precip_stats = caravan_df['p_mean'].describe()
    print(f"  Precipitation range: {precip_stats['min']:.0f} to {precip_stats['max']:.0f} mm/yr")

if 't_mean' in caravan_df.columns:
    temp_stats = caravan_df['t_mean'].describe()
    print(f"  Temperature range: {temp_stats['min']:.1f} to {temp_stats['max']:.1f} °C")

# Streamflow characteristics
if 'q_mean' in caravan_df.columns:
    flow_stats = caravan_df['q_mean'].describe()
    print(f"  Mean streamflow range: {flow_stats['min']:.1f} to {flow_stats['max']:.1f} m³/s")

# =============================================================================
# CARAVAN GLOBAL DATASET VISUALIZATION
# =============================================================================

print(f"\nCreating CARAVAN Global Dataset Overview Visualization...")

# Create comprehensive global watershed dataset overview
fig, axes = plt.subplots(2, 3, figsize=(24, 16))

# 1. Global watershed distribution map
ax1 = axes[0, 0]
if 'continent' in caravan_df.columns:
    # Color by continent
    continent_colors = {'North America': 'red', 'Europe': 'blue', 'Australia': 'green', 
                       'South America': 'orange', 'Asia': 'purple', 'Africa': 'brown'}
    
    for continent in caravan_df['continent'].unique():
        subset = caravan_df[caravan_df['continent'] == continent]
        color = continent_colors.get(continent, 'gray')
        ax1.scatter(subset['longitude'], subset['latitude'], 
                   c=color, alpha=0.7, s=30, label=continent, edgecolors='black', linewidth=0.3)
else:
    scatter = ax1.scatter(caravan_df['longitude'], caravan_df['latitude'], 
                         c=caravan_df['drainage_area'], cmap='viridis', 
                         alpha=0.7, s=30, edgecolors='black', linewidth=0.3)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title(f'CARAVAN Global Watershed Distribution\\n({len(caravan_df)} watersheds)')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-180, 180)
ax1.set_ylim(-60, 80)
if 'continent' in caravan_df.columns:
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 2. Continental distribution
ax2 = axes[0, 1]
if 'continent' in caravan_df.columns:
    continent_counts = caravan_df['continent'].value_counts()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown'][:len(continent_counts)]
    
    bars = ax2.bar(range(len(continent_counts)), continent_counts.values, 
                   color=colors, alpha=0.7, edgecolor='black')
    ax2.set_xticks(range(len(continent_counts)))
    ax2.set_xticklabels(continent_counts.index, rotation=45, ha='right')
    ax2.set_ylabel('Number of Watersheds')
    ax2.set_title('Watersheds by Continent')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, count in zip(bars, continent_counts.values):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                str(count), ha='center', va='bottom', fontweight='bold')

# 3. Climate classification
ax3 = axes[0, 2]
if 'climate_class' in caravan_df.columns:
    climate_counts = caravan_df['climate_class'].value_counts()
    colors = ['brown', 'orange', 'lightgreen', 'blue'][:len(climate_counts)]
    bars = ax3.bar(range(len(climate_counts)), climate_counts.values, 
                   color=colors, alpha=0.7, edgecolor='black')
    ax3.set_xticks(range(len(climate_counts)))
    ax3.set_xticklabels(climate_counts.index, rotation=45, ha='right')
    ax3.set_ylabel('Number of Watersheds')
    ax3.set_title('Watersheds by Climate')
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, count in zip(bars, climate_counts.values):
        ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                str(count), ha='center', va='bottom', fontweight='bold')

# 4. Flow regime distribution
ax4 = axes[1, 0]
if 'flow_regime' in caravan_df.columns:
    regime_counts = caravan_df['flow_regime'].value_counts()
    colors = ['lightblue', 'lightcoral', 'lightgreen'][:len(regime_counts)]
    bars = ax4.bar(range(len(regime_counts)), regime_counts.values,
                   color=colors, alpha=0.7, edgecolor='black')
    ax4.set_xticks(range(len(regime_counts)))
    ax4.set_xticklabels([r.replace('_', '-').title() for r in regime_counts.index], rotation=45, ha='right')
    ax4.set_ylabel('Number of Watersheds')
    ax4.set_title('Watersheds by Flow Regime')
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, count in zip(bars, regime_counts.values):
        ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                str(count), ha='center', va='bottom', fontweight='bold')

# 5. Aridity vs Temperature scatter
ax5 = axes[1, 1]
if 'aridity' in caravan_df.columns and 't_mean' in caravan_df.columns:
    scatter5 = ax5.scatter(caravan_df['t_mean'], caravan_df['aridity'], 
                          c=caravan_df['drainage_area'], cmap='viridis', 
                          alpha=0.6, s=40, edgecolors='black', linewidth=0.3)
    ax5.set_xlabel('Mean Annual Temperature (°C)')
    ax5.set_ylabel('Aridity Index')
    ax5.set_title('Climate Space: Temperature vs Aridity')
    ax5.grid(True, alpha=0.3)
    ax5.set_yscale('log')
    
    # Add climate zone boundaries
    ax5.axhline(y=3, color='red', linestyle='--', alpha=0.5, label='Arid threshold')
    ax5.axhline(y=1.5, color='orange', linestyle='--', alpha=0.5, label='Semi-arid threshold')
    ax5.legend()

# 6. Scale distribution by area
ax6 = axes[1, 2]
scale_counts = caravan_df['area_class'].value_counts()
colors = ['lightcyan', 'lightblue', 'blue', 'darkblue'][:len(scale_counts)]
bars = ax6.bar(range(len(scale_counts)), scale_counts.values,
               color=colors, alpha=0.7, edgecolor='black')
ax6.set_xticks(range(len(scale_counts)))
ax6.set_xticklabels(scale_counts.index, rotation=45, ha='right')
ax6.set_ylabel('Number of Watersheds')
ax6.set_title('Watersheds by Scale')
ax6.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, count in zip(bars, scale_counts.values):
    ax6.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
            str(count), ha='center', va='bottom', fontweight='bold')

plt.suptitle('CARAVAN Global Watershed Dataset - Comprehensive Overview', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n✅ Step 1 Complete: CARAVAN Global Dataset Analysis and Experimental Design")
print(f"   🌍 Global coverage: {len(caravan_df)} watersheds across multiple continents")
if 'continent' in caravan_df.columns:
    print(f"   🗺️  Continental representation: {', '.join(caravan_df['continent'].unique())}")
if 'climate_class' in caravan_df.columns:
    print(f"   🌡️  Climate diversity: {', '.join(caravan_df['climate_class'].unique())}")
if 'flow_regime' in caravan_df.columns:
    print(f"   🌊 Flow regimes: {', '.join(caravan_df['flow_regime'].unique())}")
print(f"   📊 Configuration template created for global streamflow analysis")

## Step 2: Automated CONFLUENCE Configuration and Global Batch Processing

Building on the global dataset analysis and configuration from Step 1, this step demonstrates 
automated large sample processing using the `run_watersheds_caravan.py` script. This script 
performs key functions for global-scale hydrological modeling:

**Global Configuration Generation**: The script reads the CARAVAN global database and automatically 
creates individual CONFLUENCE configuration files for each watershed across multiple continents. 
Each configuration is customized with site-specific parameters including domain coordinates, 
bounding box definitions, continental identifiers, and climate-specific settings, while maintaining 
consistent model settings across all global basins.

**Multi-Continental Batch Job Submission**: The script submits SLURM jobs to execute the complete 
CONFLUENCE workflow for each basin in parallel across diverse global environments. Each job processes 
geographic data, prepares meteorological forcing, processes CARAVAN observations, runs the hydrological 
model, and generates standardized output files suitable for cross-continental comparison.

This automated approach scales CONFLUENCE from regional modeling to systematic global analysis 
across 9000+ watersheds spanning multiple continents and diverse environmental conditions.


In [ ]:

# =============================================================================
# GLOBAL WATERSHED SELECTION AND CONFIGURATION
# =============================================================================

print(f"\n🌍 Step 2.1: Global Watershed Selection for CONFLUENCE Processing")

# Configuration for the global sample experiment
streamflow_config = {
    'dataset': 'caravan',
    'max_watersheds': 10,  # Start with smaller number for demonstration
    'dry_run_mode': True,  # Set to False to actually submit jobs
    'experiment_name': 'caravan_global_tutorial',
    'template_config': str(caravan_config_path),
    'config_dir': str(CONFLUENCE_CODE_DIR / '0_config_files' / 'caravan'),
    'base_data_path': str(CONFLUENCE_DATA_DIR / 'caravan'),
    'script_path': str(CONFLUENCE_CODE_DIR / 'examples' / 'run_watersheds_caravan.py')
}

# Create experiment directory structure
experiment_dir = Path(f"./experiments/{streamflow_config['experiment_name']}")
(experiment_dir / 'plots').mkdir(parents=True, exist_ok=True)
(experiment_dir / 'reports').mkdir(parents=True, exist_ok=True)
(experiment_dir / 'configs').mkdir(parents=True, exist_ok=True)

# Save configuration
with open(experiment_dir / 'global_experiment_config.yaml', 'w') as f:
    yaml.dump(streamflow_config, f, default_flow_style=False)

print(f"   📁 Experiment directory: {experiment_dir}")
print(f"   🌍 Processing scope: {streamflow_config['max_watersheds']} global watersheds")
print(f"   🗂️  Template config: {streamflow_config['template_config']}")

# Global watershed selection strategy
print(f"\n🎯 Step 2.2: Strategic Global Watershed Selection")

# Select watersheds to represent global diversity
def select_global_representative_watersheds(caravan_df, max_watersheds=10):
    """
    Select watersheds to maximize global environmental diversity
    """
    print(f"   🔍 Selecting {max_watersheds} globally representative watersheds...")
    
    selected_watersheds = []
    
    # Strategy 1: Ensure continental representation
    if 'continent' in caravan_df.columns:
        continents = caravan_df['continent'].unique()
        watersheds_per_continent = max(1, max_watersheds // len(continents))
        
        print(f"   🌍 Continental strategy: {watersheds_per_continent} watersheds per continent")
        
        for continent in continents:
            continent_data = caravan_df[caravan_df['continent'] == continent]
            
            if len(continent_data) > 0:
                # Select diverse watersheds within continent
                if len(continent_data) <= watersheds_per_continent:
                    selected = continent_data
                else:
                    # Diversify by climate and scale
                    selected = []
                    
                    # Climate diversity
                    if 'climate_class' in continent_data.columns:
                        climate_classes = continent_data['climate_class'].unique()
                        per_climate = max(1, watersheds_per_continent // len(climate_classes))
                        
                        for climate in climate_classes:
                            climate_subset = continent_data[continent_data['climate_class'] == climate]
                            if len(climate_subset) > 0:
                                # Select by different scales within climate
                                if 'area_class' in climate_subset.columns:
                                    scales = climate_subset['area_class'].unique()
                                    for scale in scales[:per_climate]:
                                        scale_subset = climate_subset[climate_subset['area_class'] == scale]
                                        if len(scale_subset) > 0:
                                            selected.append(scale_subset.iloc[0])
                                            if len(selected) >= watersheds_per_continent:
                                                break
                                    if len(selected) >= watersheds_per_continent:
                                        break
                                else:
                                    selected.extend(climate_subset.head(per_climate).to_dict('records'))
                            if len(selected) >= watersheds_per_continent:
                                break
                    else:
                        # Random selection if no climate data
                        selected = continent_data.sample(n=min(watersheds_per_continent, len(continent_data)))
                    
                    selected = pd.DataFrame(selected) if isinstance(selected, list) else selected
                
                selected_watersheds.append(selected)
                
                print(f"     {continent}: {len(selected)} watersheds selected")
    
    # Combine all selected watersheds
    if selected_watersheds:
        final_selection = pd.concat(selected_watersheds, ignore_index=True)
    else:
        # Fallback: random selection
        final_selection = caravan_df.sample(n=min(max_watersheds, len(caravan_df)))
    
    # Ensure we don't exceed max_watersheds
    if len(final_selection) > max_watersheds:
        final_selection = final_selection.head(max_watersheds)
    
    return final_selection

# Select representative watersheds
selected_watersheds = select_global_representative_watersheds(caravan_df, streamflow_config['max_watersheds'])

print(f"\n📊 Global Selection Summary:")
print(f"   Total selected: {len(selected_watersheds)} watersheds")

if 'continent' in selected_watersheds.columns:
    continent_summary = selected_watersheds['continent'].value_counts()
    print(f"   Continental distribution:")
    for continent, count in continent_summary.items():
        print(f"     {continent}: {count} watersheds")

if 'climate_class' in selected_watersheds.columns:
    climate_summary = selected_watersheds['climate_class'].value_counts()
    print(f"   Climate diversity:")
    for climate, count in climate_summary.items():
        print(f"     {climate}: {count} watersheds")

if 'flow_regime' in selected_watersheds.columns:
    regime_summary = selected_watersheds['flow_regime'].value_counts()
    print(f"   Flow regime diversity:")
    for regime, count in regime_summary.items():
        print(f"     {regime}: {count} watersheds")

# Add required columns for CONFLUENCE processing
if 'gauge_id' in selected_watersheds.columns:
    selected_watersheds['ID'] = selected_watersheds['gauge_id']
if 'gauge_lat' in selected_watersheds.columns:
    selected_watersheds['Lat'] = selected_watersheds['gauge_lat']
if 'gauge_lon' in selected_watersheds.columns:
    selected_watersheds['Lon'] = selected_watersheds['gauge_lon']
if 'area' in selected_watersheds.columns:
    selected_watersheds['Area_km2'] = selected_watersheds['area']
if 'scale' in selected_watersheds.columns:
    selected_watersheds['Scale'] = selected_watersheds['scale']

# Save selected watersheds
selected_watersheds_file = experiment_dir / 'selected_global_watersheds.csv'
selected_watersheds.to_csv(selected_watersheds_file, index=False)
print(f"   💾 Selected watersheds saved: {selected_watersheds_file}")

# =============================================================================
# GLOBAL PROCESSING VISUALIZATION
# =============================================================================

print(f"\n🗺️  Step 2.3: Global Processing Setup Visualization")

# Create global processing setup map
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Map 1: Global overview with selected watersheds
ax1 = axes[0]

# Plot all available watersheds
ax1.scatter(caravan_df['longitude'], caravan_df['latitude'], 
           c='lightgray', alpha=0.3, s=20, label='Available watersheds')

# Plot selected watersheds with continental colors
if 'continent' in selected_watersheds.columns:
    continent_colors = {
        'North America': 'red', 
        'Europe': 'blue', 
        'Australia': 'green', 
        'South America': 'orange',
        'Asia': 'purple',
        'Africa': 'brown'
    }
    
    for continent in selected_watersheds['continent'].unique():
        subset = selected_watersheds[selected_watersheds['continent'] == continent]
        color = continent_colors.get(continent, 'black')
        ax1.scatter(subset['longitude'], subset['latitude'], 
                   c=color, s=100, alpha=0.8, 
                   edgecolors='black', linewidth=2, 
                   label=f'Selected: {continent}', marker='*')

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title(f'CARAVAN Global Processing Setup\\n{len(selected_watersheds)} Selected Watersheds')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-180, 180)
ax1.set_ylim(-60, 80)
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Map 2: Selection diversity analysis
ax2 = axes[1]

# Create diversity comparison
categories = []
all_counts = []
selected_counts = []

# Continental diversity
if 'continent' in caravan_df.columns:
    all_continents = caravan_df['continent'].value_counts()
    selected_continents = selected_watersheds['continent'].value_counts()
    
    for continent in all_continents.index:
        categories.append(continent)
        all_counts.append(all_continents[continent])
        selected_counts.append(selected_continents.get(continent, 0))

# Plot diversity comparison
x_pos = np.arange(len(categories))
width = 0.35

bars1 = ax2.bar(x_pos - width/2, all_counts, width, 
               label='Available', alpha=0.6, color='lightblue')
bars2 = ax2.bar(x_pos + width/2, selected_counts, width,
               label='Selected', alpha=0.8, color='darkblue')

ax2.set_xlabel('Continent')
ax2.set_ylabel('Number of Watersheds')
ax2.set_title('Global Selection Representativeness')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(categories, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, count in zip(bars2, selected_counts):
    if count > 0:
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                str(count), ha='center', va='bottom', fontweight='bold')

plt.suptitle('CARAVAN Global Watershed Selection for CONFLUENCE Processing', 
             fontsize=16, fontweight='bold')
plt.tight_layout()

# Save the processing setup map
setup_map_path = experiment_dir / 'plots' / 'global_processing_setup.png'
plt.savefig(setup_map_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Global processing setup map saved: {setup_map_path}")

# =============================================================================
# AUTOMATED CARAVAN PROCESSING EXECUTION
# =============================================================================

def execute_caravan_global_processing():
    """
    Execute the run_watersheds_caravan.py script for global processing
    """
    print(f"\n🚀 Step 2.4: Executing CARAVAN Global Processing Script")
    
    script_path = streamflow_config['script_path']
    
    if not Path(script_path).exists():
        print(f"❌ Script not found: {script_path}")
        print(f"   📝 Expected location: {script_path}")
        print(f"   🔍 Looking for alternative locations...")
        
        # Look for the script in common locations
        possible_paths = [
            CONFLUENCE_CODE_DIR / "examples" / "run_watersheds_caravan.py",
            CONFLUENCE_CODE_DIR / "scripts" / "run_watersheds_caravan.py", 
            CONFLUENCE_CODE_DIR / "run_watersheds_caravan.py",
            Path("./run_watersheds_caravan.py")
        ]
        
        for path in possible_paths:
            if path.exists():
                script_path = str(path)
                print(f"   ✅ Found script at: {script_path}")
                break
        else:
            print(f"   ⚠️  Script not found in expected locations")
            print(f"   📋 Creating demonstration execution log...")
            return create_demonstration_processing_log()
    
    print(f"   📄 Script location: {script_path}")
    print(f"   🌍 Target watersheds: {len(selected_watersheds)} global basins")
    print(f"   🕐 Processing started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   🔧 Mode: {'DRY RUN' if streamflow_config['dry_run_mode'] else 'PRODUCTION'}")
    
    try:
        # Prepare command for CARAVAN processing
        cmd = [
            'python', script_path,
            '--dataset', streamflow_config['dataset'],
            '--template', streamflow_config['template_config'],
            '--config-dir', streamflow_config['config_dir'],
            '--max-watersheds', str(streamflow_config['max_watersheds']),
            '--watersheds-csv', str(selected_watersheds_file)
        ]
        
        if streamflow_config['dry_run_mode']:
            cmd.append('--dry-run')
        
        print(f"   💻 Command: {' '.join(cmd)}")
        
        # Execute the script
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        # Process results
        if result.returncode == 0:
            print(f"✅ CARAVAN processing script completed successfully")
            
            if result.stdout:
                print(f"\n📋 Script Output:")
                for line in result.stdout.split('\\n')[:20]:  # Show first 20 lines
                    if line.strip():
                        print(f"   {line}")
                if len(result.stdout.split('\\n')) > 20:
                    print(f"   ... (output truncated)")
            
            # Save execution log
            log_file = experiment_dir / 'processing_execution.log'
            with open(log_file, 'w') as f:
                f.write(f"CARAVAN Global Processing Execution Log\\n")
                f.write(f"{'='*50}\\n")
                f.write(f"Execution time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
                f.write(f"Command: {' '.join(cmd)}\\n")
                f.write(f"Return code: {result.returncode}\\n\\n")
                f.write("STDOUT:\\n")
                f.write(result.stdout)
                if result.stderr:
                    f.write("\\n\\nSTDERR:\\n")
                    f.write(result.stderr)
            
            print(f"   📁 Execution log saved: {log_file}")
            return True
            
        else:
            print(f"❌ Script failed with return code: {result.returncode}")
            if result.stderr:
                print(f"⚠️  Error output:")
                for line in result.stderr.split('\\n')[:10]:
                    if line.strip():
                        print(f"   {line}")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"⏰ Script execution timeout (5 minutes)")
        return False
    except Exception as e:
        print(f"❌ Error executing script: {e}")
        return False

def create_demonstration_processing_log():
    """
    Create a demonstration processing log when script is not available
    """
    print(f"   📋 Creating demonstration processing log...")
    
    # Simulate processing results
    processing_results = {
        'total_selected': len(selected_watersheds),
        'configs_generated': len(selected_watersheds),
        'jobs_submitted': len(selected_watersheds) if not streamflow_config['dry_run_mode'] else 0,
        'estimated_completion': '2-4 hours per watershed',
        'expected_outputs': [
            'Domain shapefiles',
            'Meteorological forcing',
            'SUMMA simulation results',
            'mizuRoute streamflow outputs',
            'Processed observations'
        ]
    }
    
    # Create demonstration log
    demo_log = experiment_dir / 'demonstration_processing.log'
    with open(demo_log, 'w') as f:
        f.write("CARAVAN Global Processing - Demonstration Log\\n")
        f.write("="*50 + "\\n\\n")
        f.write(f"Processing mode: {'DRY RUN' if streamflow_config['dry_run_mode'] else 'PRODUCTION'}\\n")
        f.write(f"Total watersheds selected: {processing_results['total_selected']}\\n")
        f.write(f"Configuration files to generate: {processing_results['configs_generated']}\\n")
        f.write(f"SLURM jobs to submit: {processing_results['jobs_submitted']}\\n")
        f.write(f"Estimated processing time: {processing_results['estimated_completion']}\\n\\n")
        
        f.write("Expected outputs per watershed:\\n")
        for output in processing_results['expected_outputs']:
            f.write(f"  - {output}\\n")
        
        f.write("\\nProcessing workflow:\\n")
        f.write("  1. Generate domain-specific CONFLUENCE configurations\\n")
        f.write("  2. Download and process geographic data\\n")
        f.write("  3. Prepare meteorological forcing data\\n")
        f.write("  4. Process CARAVAN streamflow observations\\n")
        f.write("  5. Execute SUMMA hydrological modeling\\n")
        f.write("  6. Run mizuRoute streamflow routing\\n")
        f.write("  7. Generate standardized output files\\n")
    
    print(f"   📄 Demonstration log created: {demo_log}")
    
    # Display processing summary
    print(f"\\n📊 Global Processing Summary:")
    print(f"   🌍 Watersheds: {processing_results['total_selected']} across multiple continents")
    print(f"   ⚙️  Configurations: {processing_results['configs_generated']} to be generated")
    print(f"   🖥️  Jobs: {processing_results['jobs_submitted']} {'(dry run)' if streamflow_config['dry_run_mode'] else 'to submit'}")
    print(f"   ⏱️  Estimated time: {processing_results['estimated_completion']}")
    
    return True

# Execute the global processing
processing_success = execute_caravan_global_processing()

# =============================================================================
# PROCESSING STATUS AND MONITORING
# =============================================================================

print(f"\\n📈 Step 2.5: Global Processing Status and Monitoring")

def create_processing_status_summary():
    """
    Create comprehensive processing status summary
    """
    
    status_summary = {
        'experiment_name': streamflow_config['experiment_name'],
        'processing_mode': 'DRY RUN' if streamflow_config['dry_run_mode'] else 'PRODUCTION',
        'total_watersheds': len(selected_watersheds),
        'script_executed': processing_success,
        'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Continental breakdown
    if 'continent' in selected_watersheds.columns:
        continental_breakdown = selected_watersheds['continent'].value_counts().to_dict()
        status_summary['continental_breakdown'] = continental_breakdown
    
    # Climate breakdown
    if 'climate_class' in selected_watersheds.columns:
        climate_breakdown = selected_watersheds['climate_class'].value_counts().to_dict()
        status_summary['climate_breakdown'] = climate_breakdown
    
    # Scale breakdown
    if 'area_class' in selected_watersheds.columns:
        scale_breakdown = selected_watersheds['area_class'].value_counts().to_dict()
        status_summary['scale_breakdown'] = scale_breakdown
    
    # Expected outputs
    status_summary['expected_outputs'] = {
        'domain_directories': len(selected_watersheds),
        'shapefile_sets': len(selected_watersheds),
        'forcing_datasets': len(selected_watersheds),
        'simulation_results': len(selected_watersheds),
        'streamflow_outputs': len(selected_watersheds),
        'observation_files': len(selected_watersheds)
    }
    
    # Save status summary
    status_file = experiment_dir / 'processing_status_summary.yaml'
    with open(status_file, 'w') as f:
        yaml.dump(status_summary, f, default_flow_style=False)
    
    print(f"   📊 Processing status summary:")
    print(f"     Experiment: {status_summary['experiment_name']}")
    print(f"     Mode: {status_summary['processing_mode']}")
    print(f"     Watersheds: {status_summary['total_watersheds']}")
    print(f"     Script executed: {status_summary['script_executed']}")
    
    if 'continental_breakdown' in status_summary:
        print(f"     Continental distribution:")
        for continent, count in status_summary['continental_breakdown'].items():
            print(f"       {continent}: {count} watersheds")
    
    print(f"   💾 Status summary saved: {status_file}")
    
    return status_summary

# Create processing status summary
processing_status = create_processing_status_summary()

print(f"\\n✅ Step 2 Complete: CARAVAN Global Processing Setup and Execution")
print(f"   🌍 Global scope: {len(selected_watersheds)} watersheds across continents")
print(f"   ⚙️  Configuration: Template and processing scripts prepared")
print(f"   🚀 Execution: {'Completed' if processing_success else 'Attempted'}")
print(f"   📁 Results: All outputs saved to {experiment_dir}")

if streamflow_config['dry_run_mode']:
    print(f"   🔧 Mode: DRY RUN - Switch to production mode to submit actual jobs")
else:
    print(f"   🔧 Mode: PRODUCTION - Jobs submitted for processing")

print(f"\\n🎯 Next: Proceed to Step 3 for global streamflow validation and analysis")

## Step 3: Global Multi-Basin Streamflow Validation and Continental Analysis

Having executed large sample global streamflow modeling, we now demonstrate the analytical power 
that emerges from systematic multi-continental streamflow validation using CARAVAN observations. 
This step showcases comprehensive watershed response evaluation across continents, continental 
performance assessment, and integrated global process validation—the scientific culmination of 
our entire CONFLUENCE tutorial series.

In [ ]:
import xarray as xr

def discover_completed_global_streamflow_domains():
    """
    Discover all completed CARAVAN domain directories and their streamflow outputs across continents
    """
    print(f"\n🔍 Discovering Completed CARAVAN Global Streamflow Modeling Domains...")
    
    # Base data directory pattern
    base_path = Path(streamflow_config['base_data_path'])
    domain_pattern = str(base_path / "domain_*")
    
    # Find all domain directories
    domain_dirs = glob.glob(domain_pattern)
    
    print(f"   📁 Found {len(domain_dirs)} total domain directories")
    
    completed_domains = []
    
    for domain_dir in domain_dirs:
        domain_path = Path(domain_dir)
        domain_name = domain_path.name.replace('domain_', '')
        
        # Check if this is a CARAVAN domain (should match our selected watersheds)
        if any(domain_name.startswith(ws) for ws in selected_watersheds['ID'].values):
            
            # Check for key output files
            shapefile_path = domain_path / "shapefiles" / "river_basins"
            simulation_dir = domain_path / "simulations"
            obs_dir = domain_path / "observations" / "streamflow" / "preprocessed"
            
            domain_info = {
                'domain_name': domain_name,
                'domain_path': domain_path,
                'has_shapefile': shapefile_path.exists(),
                'shapefile_path': shapefile_path if shapefile_path.exists() else None,
                'has_simulations': simulation_dir.exists(),
                'simulation_path': simulation_dir if simulation_dir.exists() else None,
                'has_observations': obs_dir.exists(),
                'observation_path': obs_dir if obs_dir.exists() else None,
                'simulation_files': [],
                'streamflow_obs_file': None
            }
            
            # Find simulation output files
            if simulation_dir.exists():
                # Look for SUMMA outputs
                summa_files = list(simulation_dir.glob("**/SUMMA/*.nc"))
                # Look for mizuRoute outputs (streamflow routing)
                mizuroute_files = list(simulation_dir.glob("**/mizuRoute/*.nc"))
                
                domain_info['simulation_files'] = summa_files + mizuroute_files
                domain_info['has_results'] = len(domain_info['simulation_files']) > 0
                domain_info['has_summa'] = len(summa_files) > 0
                domain_info['has_routing'] = len(mizuroute_files) > 0
            else:
                domain_info['has_results'] = False
                domain_info['has_summa'] = False
                domain_info['has_routing'] = False
            
            # Find observation files
            if obs_dir.exists():
                streamflow_files = list(obs_dir.glob("*streamflow*.csv"))
                if streamflow_files:
                    domain_info['streamflow_obs_file'] = streamflow_files[0]
            
            # Add continental information
            watershed_row = None
            for _, row in selected_watersheds.iterrows():
                if domain_name.startswith(row['ID']):
                    watershed_row = row
                    break
            
            if watershed_row is not None:
                domain_info['continent'] = watershed_row.get('continent', 'Unknown')
                domain_info['climate_class'] = watershed_row.get('climate_class', 'Unknown')
                domain_info['flow_regime'] = watershed_row.get('flow_regime', 'Unknown')
                domain_info['watershed_scale'] = watershed_row.get('area_class', 'Unknown')
            
            completed_domains.append(domain_info)
    
    print(f"   🌍 CARAVAN global domains found: {len(completed_domains)}")
    print(f"   📊 Domains with shapefiles: {sum(1 for d in completed_domains if d['has_shapefile'])}")
    print(f"   📈 Domains with simulation results: {sum(1 for d in completed_domains if d['has_results'])}")
    print(f"   🌊 Domains with routing outputs: {sum(1 for d in completed_domains if d['has_routing'])}")
    print(f"   📋 Domains with observations: {sum(1 for d in completed_domains if d['has_observations'])}")
    
    # Continental breakdown
    if completed_domains:
        continental_summary = {}
        for domain in completed_domains:
            continent = domain.get('continent', 'Unknown')
            if continent not in continental_summary:
                continental_summary[continent] = {'total': 0, 'with_results': 0, 'with_routing': 0}
            continental_summary[continent]['total'] += 1
            if domain['has_results']:
                continental_summary[continent]['with_results'] += 1
            if domain['has_routing']:
                continental_summary[continent]['with_routing'] += 1
        
        print(f"   🗺️  Continental breakdown:")
        for continent, stats in continental_summary.items():
            print(f"     {continent}: {stats['total']} total, {stats['with_results']} with results, {stats['with_routing']} with routing")
    
    return completed_domains

def create_global_streamflow_domain_overview_map(completed_domains):
    """
    Create an overview map showing all global streamflow domain locations and their completion status
    """
    print(f"\n🗺️  Creating Global Streamflow Domain Overview Map...")
    
    # Create figure for global overview map
    fig, axes = plt.subplots(2, 2, figsize=(24, 16))
    
    # Map 1: Global overview with completion status
    ax1 = axes[0, 0]
    
    # Plot all selected sites
    if len(selected_watersheds) > 0:
        ax1.scatter(selected_watersheds['Lon'], selected_watersheds['Lat'], 
                   c='lightgray', alpha=0.5, s=40, label='Selected watersheds', marker='o')
    
    # Plot completed domains with different colors for different completion levels
    continent_colors = {'North America': 'red', 'Europe': 'blue', 'Australia': 'green', 
                       'South America': 'orange', 'Asia': 'purple', 'Africa': 'brown'}
    
    for domain in completed_domains:
        domain_name = domain['domain_name']
        
        # Find corresponding site in selected_watersheds
        site_row = None
        for _, row in selected_watersheds.iterrows():
            if domain_name.startswith(row['ID']):
                site_row = row
                break
        
        if site_row is not None:
            lat = site_row['Lat']
            lon = site_row['Lon']
            continent = domain.get('continent', 'Unknown')
            base_color = continent_colors.get(continent, 'gray')
            
            # Marker style based on completion status
            if domain['has_routing'] and domain['has_observations']:
                marker = 's'
                size = 120
                alpha = 1.0
                label = 'Complete with streamflow validation'
            elif domain['has_routing']:
                marker = '^'
                size = 100
                alpha = 0.8
                label = 'Routing complete'
            elif domain['has_results']:
                marker = 'D'
                size = 80
                alpha = 0.7
                label = 'Simulation complete'
            else:
                marker = 'v'
                size = 60
                alpha = 0.5
                label = 'Processing started'
            
            ax1.scatter(lon, lat, c=base_color, s=size, marker=marker, alpha=alpha,
                       edgecolors='black', linewidth=1, label=f'{continent} - {label}')
    
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('CARAVAN Global Streamflow Domain Processing Status Overview')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-180, 180)
    ax1.set_ylim(-60, 80)
    
    # Create custom legend
    legend_elements = []
    for continent, color in continent_colors.items():
        if any(d.get('continent') == continent for d in completed_domains):
            legend_elements.append(plt.scatter([], [], c=color, s=60, label=continent))
    
    # Add completion status legend
    legend_elements.extend([
        plt.scatter([], [], c='gray', s=120, marker='s', label='Complete with validation'),
        plt.scatter([], [], c='gray', s=100, marker='^', label='Routing complete'),
        plt.scatter([], [], c='gray', s=80, marker='D', label='Simulation complete'),
        plt.scatter([], [], c='gray', s=60, marker='v', label='Processing started')
    ])
    
    ax1.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Map 2: Continental completion statistics
    ax2 = axes[0, 1]
    
    # Create continental completion analysis
    continental_completion = {}
    
    for domain in completed_domains:
        continent = domain.get('continent', 'Unknown')
        
        if continent not in continental_completion:
            continental_completion[continent] = {'total': 0, 'complete': 0, 'partial': 0, 'pending': 0}
        
        continental_completion[continent]['total'] += 1
        
        if domain['has_routing'] and domain['has_observations']:
            continental_completion[continent]['complete'] += 1
        elif domain['has_results']:
            continental_completion[continent]['partial'] += 1
        else:
            continental_completion[continent]['pending'] += 1
    
    # Create stacked bar chart
    if continental_completion:
        continents = list(continental_completion.keys())
        complete_counts = [continental_completion[c]['complete'] for c in continents]
        partial_counts = [continental_completion[c]['partial'] for c in continents]
        pending_counts = [continental_completion[c]['pending'] for c in continents]
        
        x_pos = range(len(continents))
        
        ax2.bar(x_pos, complete_counts, label='Complete', color='green', alpha=0.8)
        ax2.bar(x_pos, partial_counts, bottom=complete_counts, 
               label='Partial', color='orange', alpha=0.8)
        ax2.bar(x_pos, pending_counts, 
               bottom=[c+p for c,p in zip(complete_counts, partial_counts)], 
               label='Pending', color='red', alpha=0.8)
        
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels(continents, rotation=45, ha='right')
        ax2.set_ylabel('Number of Watersheds')
        ax2.set_title('Processing Status by Continent')
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')
    
    # Map 3: Climate zone vs completion status
    ax3 = axes[1, 0]
    
    climate_completion = {}
    for domain in completed_domains:
        climate = domain.get('climate_class', 'Unknown')
        if climate not in climate_completion:
            climate_completion[climate] = {'complete': 0, 'partial': 0, 'pending': 0}
        
        if domain['has_routing'] and domain['has_observations']:
            climate_completion[climate]['complete'] += 1
        elif domain['has_results']:
            climate_completion[climate]['partial'] += 1
        else:
            climate_completion[climate]['pending'] += 1
    
    if climate_completion:
        climates = list(climate_completion.keys())
        complete_counts = [climate_completion[c]['complete'] for c in climates]
        partial_counts = [climate_completion[c]['partial'] for c in climates]
        pending_counts = [climate_completion[c]['pending'] for c in climates]
        
        x_pos = range(len(climates))
        
        ax3.bar(x_pos, complete_counts, label='Complete', color='green', alpha=0.8)
        ax3.bar(x_pos, partial_counts, bottom=complete_counts, 
               label='Partial', color='orange', alpha=0.8)
        ax3.bar(x_pos, pending_counts, 
               bottom=[c+p for c,p in zip(complete_counts, partial_counts)], 
               label='Pending', color='red', alpha=0.8)
        
        ax3.set_xticks(x_pos)
        ax3.set_xticklabels(climates, rotation=45, ha='right')
        ax3.set_ylabel('Number of Watersheds')
        ax3.set_title('Processing Status by Climate Zone')
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
    
    # Map 4: Global processing summary statistics
    ax4 = axes[1, 1]
    
    # Summary statistics
    total_selected = len(selected_watersheds) if len(selected_watersheds) > 0 else 0
    total_discovered = len(completed_domains)
    total_with_results = sum(1 for d in completed_domains if d['has_results'])
    total_with_routing = sum(1 for d in completed_domains if d['has_routing'])
    total_with_obs = sum(1 for d in completed_domains if d['has_observations'])
    total_complete = sum(1 for d in completed_domains if d['has_routing'] and d['has_observations'])
    
    categories = ['Selected\\nGlobally', 'Processing\\nStarted', 'Simulation\\nComplete', 
                 'Routing\\nComplete', 'Observations\\nAvailable', 'Ready for\\nValidation']
    counts = [total_selected, total_discovered, total_with_results, total_with_routing, total_with_obs, total_complete]
    colors = ['lightblue', 'yellow', 'blue', 'orange', 'cyan', 'green']
    
    bars = ax4.bar(categories, counts, color=colors, alpha=0.8, edgecolor='black')
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                str(count), ha='center', va='bottom', fontweight='bold')
    
    ax4.set_ylabel('Number of Watersheds')
    ax4.set_title('Global Streamflow Modeling Processing Progress')
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle('CARAVAN Global Large Sample Streamflow Study - Domain Overview', 
                 fontsize=18, fontweight='bold')
    plt.tight_layout()
    
    # Save the overview map
    overview_path = experiment_dir / 'plots' / 'global_streamflow_domain_overview_map.png'
    plt.savefig(overview_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Global streamflow domain overview map saved: {overview_path}")
    
    return total_selected, total_discovered, total_with_results, total_with_routing, total_with_obs, total_complete

def extract_global_streamflow_results_from_domains(completed_domains):
    """
    Extract streamflow simulation results from all completed global domains
    """
    print(f"\n🌊 Extracting Global Streamflow Results from Completed Domains...")
    
    streamflow_results = []
    processing_summary = {
        'total_domains': len(completed_domains),
        'domains_with_routing': 0,
        'domains_with_streamflow': 0,
        'failed_extractions': 0,
        'continental_breakdown': {}
    }
    
    for domain in completed_domains:
        if not domain['has_routing']:
            continue
            
        domain_name = domain['domain_name']
        continent = domain.get('continent', 'Unknown')
        processing_summary['domains_with_routing'] += 1
        
        if continent not in processing_summary['continental_breakdown']:
            processing_summary['continental_breakdown'][continent] = {'attempted': 0, 'successful': 0}
        processing_summary['continental_breakdown'][continent]['attempted'] += 1
        
        try:
            print(f"   🔄 Processing {domain_name} ({continent})...")
            
            # Find routing output files (mizuRoute)
            mizuroute_files = [f for f in domain['simulation_files'] if 'mizuRoute' in str(f)]
            
            if not mizuroute_files:
                print(f"     ❌ No mizuRoute files found")
                processing_summary['failed_extractions'] += 1
                continue
            
            # Use the first mizuRoute file
            output_file = mizuroute_files[0]
            
            # Load the netCDF file
            ds = xr.open_dataset(output_file)
            
            # Look for streamflow variables
            streamflow_vars = {}
            
            # Common mizuRoute streamflow variable names
            potential_vars = ['IRFroutedRunoff', 'routedRunoff', 'discharge', 'streamflow']
            
            for var in potential_vars:
                if var in ds.data_vars:
                    streamflow_vars['discharge'] = var
                    break
            
            if not streamflow_vars:
                print(f"     ⚠️  No streamflow variables found in {output_file.name}")
                available_vars = list(ds.data_vars.keys())
                print(f"     Available variables: {available_vars[:5]}...")
                processing_summary['failed_extractions'] += 1
                continue
            
            print(f"     🌊 Using streamflow variable: {streamflow_vars['discharge']}")
            
            # Extract streamflow data
            streamflow_var = streamflow_vars['discharge']
            streamflow_data = ds[streamflow_var]
            
            # Handle multi-dimensional data (time x reaches)
            if len(streamflow_data.dims) > 1:
                # Find the time dimension
                time_dim = 'time'
                reach_dims = [dim for dim in streamflow_data.dims if dim != time_dim]
                
                if reach_dims:
                    reach_dim = reach_dims[0]
                    # Use the last reach (often the outlet)
                    outlet_idx = streamflow_data.sizes[reach_dim] - 1
                    streamflow_data = streamflow_data.isel({reach_dim: outlet_idx})
                    print(f"     📍 Using outlet reach (index {outlet_idx})")
            
            # Convert to pandas Series
            streamflow_series = streamflow_data.to_pandas()
            
            # Handle unit conversion if needed (assume m³/s is correct)
            # Remove any negative values (set to 0)
            streamflow_series = streamflow_series.clip(lower=0)
            
            # Get site information
            site_row = None
            for _, row in selected_watersheds.iterrows():
                if domain_name.startswith(row['ID']):
                    site_row = row
                    break
            
            if site_row is None:
                print(f"     ⚠️  Site information not found for {domain_name}")
                continue
            
            # Calculate streamflow statistics
            streamflow_stats = {
                'mean_flow': streamflow_series.mean(),
                'max_flow': streamflow_series.max(),
                'min_flow': streamflow_series.min(),
                'std_flow': streamflow_series.std(),
                'flow_variability': streamflow_series.std() / streamflow_series.mean() if streamflow_series.mean() > 0 else np.nan
            }
            
            # Calculate flow percentiles
            percentiles = [5, 25, 50, 75, 95]
            for p in percentiles:
                streamflow_stats[f'q{p}'] = streamflow_series.quantile(p/100)
            
            # Store results
            result = {
                'domain_name': domain_name,
                'watershed_id': site_row['ID'],
                'latitude': site_row['Lat'],
                'longitude': site_row['Lon'],
                'area_km2': site_row.get('Area_km2', np.nan),
                'scale': site_row.get('Scale', 'unknown'),
                'continent': continent,
                'climate_class': domain.get('climate_class', 'Unknown'),
                'flow_regime': domain.get('flow_regime', 'Unknown'),
                'streamflow_timeseries': streamflow_series,
                'data_period': f"{streamflow_series.index.min()} to {streamflow_series.index.max()}",
                'data_points': len(streamflow_series),
                'streamflow_variable': streamflow_var,
                'output_file': str(output_file)
            }
            
            # Add statistics
            result.update(streamflow_stats)
            
            streamflow_results.append(result)
            processing_summary['domains_with_streamflow'] += 1
            processing_summary['continental_breakdown'][continent]['successful'] += 1
            
            print(f"     ✅ Streamflow extracted: {result['mean_flow']:.2f} m³/s (range: {result['min_flow']:.2f}-{result['max_flow']:.2f})")
            
        except Exception as e:
            print(f"     ❌ Error processing {domain_name}: {e}")
            processing_summary['failed_extractions'] += 1
    
    print(f"\n🌊 Global Streamflow Extraction Summary:")
    print(f"   Total domains: {processing_summary['total_domains']}")
    print(f"   Domains with routing: {processing_summary['domains_with_routing']}")
    print(f"   Successful extractions: {processing_summary['domains_with_streamflow']}")
    print(f"   Failed extractions: {processing_summary['failed_extractions']}")
    
    print(f"   🗺️  Continental breakdown:")
    for continent, stats in processing_summary['continental_breakdown'].items():
        success_rate = (stats['successful'] / stats['attempted'] * 100) if stats['attempted'] > 0 else 0
        print(f"     {continent}: {stats['successful']}/{stats['attempted']} ({success_rate:.0f}% success)")
    
    return streamflow_results, processing_summary

def load_caravan_global_observations(completed_domains):
    """
    Load CARAVAN observation data for global streamflow validation
    """
    print(f"\n📥 Loading CARAVAN Global Streamflow Observation Data...")
    
    caravan_obs = {}
    obs_summary = {
        'sites_found': 0,
        'sites_with_streamflow': 0,
        'total_observations': 0,
        'continental_breakdown': {}
    }
    
    # Look for processed CARAVAN observation data in domain directories
    for domain in completed_domains:
        if not domain['has_observations']:
            continue
            
        domain_name = domain['domain_name']
        continent = domain.get('continent', 'Unknown')
        
        if continent not in obs_summary['continental_breakdown']:
            obs_summary['continental_breakdown'][continent] = {'found': 0, 'with_streamflow': 0}
        
        try:
            print(f"   📊 Loading {domain_name} ({continent})...")
            
            obs_summary['sites_found'] += 1
            obs_summary['continental_breakdown'][continent]['found'] += 1
            
            # Load streamflow observations
            if domain['streamflow_obs_file']:
                obs_df = pd.read_csv(domain['streamflow_obs_file'])
                
                # Find time and discharge columns
                time_col = None
                for col in ['datetime', 'date', 'time']:
                    if col in obs_df.columns:
                        time_col = col
                        break
                
                discharge_col = None
                for col in ['discharge_cms', 'streamflow', 'flow', 'Q']:
                    if col in obs_df.columns:
                        discharge_col = col
                        break
                
                if time_col and discharge_col:
                    obs_df[time_col] = pd.to_datetime(obs_df[time_col])
                    obs_df.set_index(time_col, inplace=True)
                    
                    streamflow_obs = obs_df[discharge_col].dropna()
                    
                    if len(streamflow_obs) > 0:
                        # Calculate streamflow statistics
                        obs_stats = {
                            'mean_flow': streamflow_obs.mean(),
                            'max_flow': streamflow_obs.max(),
                            'min_flow': streamflow_obs.min(),
                            'std_flow': streamflow_obs.std(),
                            'flow_variability': streamflow_obs.std() / streamflow_obs.mean() if streamflow_obs.mean() > 0 else np.nan
                        }
                        
                        # Calculate flow percentiles
                        percentiles = [5, 25, 50, 75, 95]
                        for p in percentiles:
                            obs_stats[f'q{p}'] = streamflow_obs.quantile(p/100)
                        
                        # Store observation data
                        site_obs = {
                            'streamflow_timeseries': streamflow_obs,
                            'data_period': f"{streamflow_obs.index.min()} to {streamflow_obs.index.max()}",
                            'data_points': len(streamflow_obs),
                            'continent': continent,
                            'climate_class': domain.get('climate_class', 'Unknown'),
                            'flow_regime': domain.get('flow_regime', 'Unknown')
                        }
                        
                        # Add statistics
                        site_obs.update(obs_stats)
                        
                        # Add site metadata
                        site_row = None
                        for _, row in selected_watersheds.iterrows():
                            if domain_name.startswith(row['ID']):
                                site_row = row
                                break
                        
                        if site_row is not None:
                            site_obs['latitude'] = site_row['Lat']
                            site_obs['longitude'] = site_row['Lon']
                            site_obs['area_km2'] = site_row.get('Area_km2', np.nan)
                            site_obs['scale'] = site_row.get('Scale', 'unknown')
                            site_obs['watershed_id'] = site_row['ID']
                        
                        caravan_obs[domain_name] = site_obs
                        
                        obs_summary['sites_with_streamflow'] += 1
                        obs_summary['continental_breakdown'][continent]['with_streamflow'] += 1
                        obs_summary['total_observations'] += len(streamflow_obs)
                        
                        print(f"     🌊 Streamflow obs: {streamflow_obs.mean():.2f} m³/s (range: {streamflow_obs.min():.2f}-{streamflow_obs.max():.2f}) ({len(streamflow_obs)} points)")
                else:
                    print(f"     ⚠️ Could not find time/discharge columns in observation file")
            
        except Exception as e:
            print(f"     ❌ Error loading {domain_name}: {e}")
    
    print(f"\n🌊 CARAVAN Global Observation Summary:")
    print(f"   Sites with observation files: {obs_summary['sites_found']}")
    print(f"   Sites with streamflow observations: {obs_summary['sites_with_streamflow']}")
    print(f"   Total streamflow observations: {obs_summary['total_observations']}")
    
    print(f"   🗺️  Continental breakdown:")
    for continent, stats in obs_summary['continental_breakdown'].items():
        print(f"     {continent}: {stats['with_streamflow']}/{stats['found']} sites with streamflow")
    
    return caravan_obs, obs_summary

def create_global_streamflow_comparison_analysis(streamflow_results, caravan_obs):
    """
    Create comprehensive global streamflow comparison analysis between simulated and observed
    """
    print(f"\n🌊 Creating Global Streamflow Comparison Analysis...")
    
    # Find sites with both simulated and observed data
    common_sites = []
    
    for sim_result in streamflow_results:
        domain_name = sim_result['domain_name']
        
        if domain_name in caravan_obs:
            # Align time periods
            sim_flow = sim_result['streamflow_timeseries']
            obs_flow = caravan_obs[domain_name]['streamflow_timeseries']
            
            # Find common time period
            common_start = max(sim_flow.index.min(), obs_flow.index.min())
            common_end = min(sim_flow.index.max(), obs_flow.index.max())
            
            if common_start < common_end:
                # Resample to daily and align
                sim_daily = sim_flow.resample('D').mean().loc[common_start:common_end]
                obs_daily = obs_flow.resample('D').mean().loc[common_start:common_end]
                
                # Remove NaN values
                valid_mask = ~(sim_daily.isna() | obs_daily.isna())
                sim_valid = sim_daily[valid_mask]
                obs_valid = obs_daily[valid_mask]
                
                if len(sim_valid) > 50:  # Need minimum data for meaningful comparison
                    
                    # Calculate performance metrics
                    def calculate_nse(obs, sim):
                        return 1 - ((obs - sim) ** 2).sum() / ((obs - obs.mean()) ** 2).sum()
                    
                    def calculate_kge(obs, sim):
                        # Kling-Gupta Efficiency
                        r = np.corrcoef(obs, sim)[0, 1]
                        alpha = sim.std() / obs.std()
                        beta = sim.mean() / obs.mean()
                        kge = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
                        return kge
                    
                    # Performance metrics
                    nse = calculate_nse(obs_valid, sim_valid)
                    rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
                    bias = (sim_valid - obs_valid).mean()
                    pbias = 100 * bias / obs_valid.mean() if obs_valid.mean() > 0 else np.nan
                    
                    # Correlation
                    try:
                        correlation = obs_valid.corr(sim_valid)
                        if pd.isna(correlation):
                            correlation = 0.0
                    except:
                        correlation = 0.0
                    
                    # KGE
                    try:
                        kge = calculate_kge(obs_valid.values, sim_valid.values)
                        if pd.isna(kge):
                            kge = -999
                    except:
                        kge = -999
                    
                    common_site = {
                        'domain_name': domain_name,
                        'watershed_id': sim_result['watershed_id'],
                        'latitude': sim_result['latitude'],
                        'longitude': sim_result['longitude'],
                        'area_km2': sim_result['area_km2'],
                        'scale': sim_result['scale'],
                        'continent': sim_result['continent'],
                        'climate_class': sim_result['climate_class'],
                        'flow_regime': sim_result['flow_regime'],
                        'sim_flow': sim_valid,
                        'obs_flow': obs_valid,
                        'sim_mean': sim_valid.mean(),
                        'obs_mean': obs_valid.mean(),
                        'nse': nse,
                        'kge': kge,
                        'rmse': rmse,
                        'bias': bias,
                        'pbias': pbias,
                        'correlation': correlation,
                        'n_points': len(sim_valid),
                        'common_period': f"{common_start.date()} to {common_end.date()}"
                    }
                    
                    common_sites.append(common_site)
                    
                    print(f"   ✅ {domain_name} ({sim_result['continent']}): NSE={nse:.3f}, KGE={kge:.3f}, r={correlation:.3f} ({len(sim_valid)} points)")
    
    print(f"\n🌊 Global Streamflow Comparison Summary:")
    print(f"   Sites with both sim and obs: {len(common_sites)}")
    
    if len(common_sites) == 0:
        print(f"   ⚠️  No sites with overlapping sim/obs data for comparison")
        return None
    
    # Continental breakdown
    continental_performance = {}
    for site in common_sites:
        continent = site['continent']
        if continent not in continental_performance:
            continental_performance[continent] = {'count': 0, 'nse_sum': 0, 'kge_sum': 0}
        continental_performance[continent]['count'] += 1
        continental_performance[continent]['nse_sum'] += site['nse']
        if site['kge'] != -999:
            continental_performance[continent]['kge_sum'] += site['kge']
    
    print(f"   🗺️  Continental performance:")
    for continent, stats in continental_performance.items():
        mean_nse = stats['nse_sum'] / stats['count']
        mean_kge = stats['kge_sum'] / stats['count'] if stats['count'] > 0 else 0
        print(f"     {continent}: {stats['count']} sites, Mean NSE={mean_nse:.3f}, Mean KGE={mean_kge:.3f}")
    
    # Create comprehensive global streamflow comparison visualization
    fig, axes = plt.subplots(3, 3, figsize=(24, 18))
    
    # Scatter plot: Observed vs Simulated (top left)
    ax1 = axes[0, 0]
    
    all_obs = np.concatenate([site['obs_flow'].values for site in common_sites])
    all_sim = np.concatenate([site['sim_flow'].values for site in common_sites])
    
    ax1.scatter(all_obs, all_sim, alpha=0.3, s=8, c='blue')
    
    # 1:1 line
    min_val = min(all_obs.min(), all_sim.min())
    max_val = max(all_obs.max(), all_sim.max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'k--', label='1:1 line')
    
    ax1.set_xlabel('Observed Streamflow (m³/s)')
    ax1.set_ylabel('Simulated Streamflow (m³/s)')
    ax1.set_title('Global: Simulated vs Observed Streamflow')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    
    # Add overall statistics
    overall_corr = np.corrcoef(all_obs, all_sim)[0,1] if len(all_obs) > 1 else 0
    overall_nse = 1 - ((all_obs - all_sim) ** 2).sum() / ((all_obs - all_obs.mean()) ** 2).sum()
    overall_bias = np.mean(all_sim - all_obs)
    
    stats_text = f'r = {overall_corr:.3f}\\nNSE = {overall_nse:.3f}\\nBias = {overall_bias:+.2f}'
    ax1.text(0.05, 0.95, stats_text, transform=ax1.transAxes,
             bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Performance by continent (top middle)
    ax2 = axes[0, 1]
    
    continental_stats = {}
    for site in common_sites:
        continent = site['continent']
        if continent not in continental_stats:
            continental_stats[continent] = {'nse': [], 'kge': [], 'corr': []}
        
        continental_stats[continent]['nse'].append(site['nse'])
        continental_stats[continent]['kge'].append(site['kge'])
        continental_stats[continent]['corr'].append(site['correlation'])
    
    # Plot NSE by continent
    continents = list(continental_stats.keys())
    nse_means = [np.mean(continental_stats[c]['nse']) for c in continents]
    nse_stds = [np.std(continental_stats[c]['nse']) for c in continents]
    
    continent_colors = {'North America': 'red', 'Europe': 'blue', 'Australia': 'green', 
                       'South America': 'orange', 'Asia': 'purple', 'Africa': 'brown'}
    colors = [continent_colors.get(c, 'gray') for c in continents]
    
    x_pos = range(len(continents))
    bars = ax2.bar(x_pos, nse_means, yerr=nse_stds, capsize=5, alpha=0.7, color=colors)
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(continents, rotation=45, ha='right')
    ax2.set_ylabel('Nash-Sutcliffe Efficiency')
    ax2.set_title('Streamflow Performance by Continent')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    
    # Add value labels
    for bar, mean_val in zip(bars, nse_means):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.02,
                f'{mean_val:.2f}', ha='center', va='bottom', fontsize=9)
    
    # Performance by climate zone (top right)
    ax3 = axes[0, 2]
    
    climate_stats = {}
    for site in common_sites:
        climate = site['climate_class']
        if climate not in climate_stats:
            climate_stats[climate] = {'nse': [], 'kge': []}
        
        climate_stats[climate]['nse'].append(site['nse'])
        climate_stats[climate]['kge'].append(site['kge'])
    
    if climate_stats:
        climates = list(climate_stats.keys())
        nse_means = [np.mean(climate_stats[c]['nse']) for c in climates]
        nse_stds = [np.std(climate_stats[c]['nse']) for c in climates]
        
        climate_colors = {'Arid': 'brown', 'Semi-arid': 'orange', 'Sub-humid': 'lightgreen', 'Humid': 'blue'}
        colors = [climate_colors.get(c, 'gray') for c in climates]
        
        x_pos = range(len(climates))
        bars = ax3.bar(x_pos, nse_means, yerr=nse_stds, capsize=5, alpha=0.7, color=colors)
        ax3.set_xticks(x_pos)
        ax3.set_xticklabels(climates, rotation=45, ha='right')
        ax3.set_ylabel('Nash-Sutcliffe Efficiency')
        ax3.set_title('Performance by Climate Zone')
        ax3.grid(True, alpha=0.3, axis='y')
        ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    
    # Global spatial distribution of NSE (middle left)
    ax4 = axes[1, 0]
    
    lats = [site['latitude'] for site in common_sites]
    lons = [site['longitude'] for site in common_sites]
    nse_values = [site['nse'] for site in common_sites]
    
    scatter4 = ax4.scatter(lons, lats, c=nse_values, cmap='RdYlGn', s=100, 
                          vmin=-0.5, vmax=1.0, edgecolors='black', linewidth=0.5)
    
    ax4.set_xlabel('Longitude')
    ax4.set_ylabel('Latitude')
    ax4.set_title('Global Distribution: NSE Performance')
    ax4.grid(True, alpha=0.3)
    ax4.set_xlim(-180, 180)
    ax4.set_ylim(-60, 80)
    
    # Add colorbar
    cbar4 = plt.colorbar(scatter4, ax=ax4)
    cbar4.set_label('Nash-Sutcliffe Efficiency')
    
    # Flow regime performance (middle center)
    ax5 = axes[1, 1]
    
    regime_stats = {}
    for site in common_sites:
        regime = site['flow_regime']
        if regime not in regime_stats:
            regime_stats[regime] = {'nse': [], 'count': 0}
        
        regime_stats[regime]['nse'].append(site['nse'])
        regime_stats[regime]['count'] += 1
    
    if regime_stats:
        regimes = list(regime_stats.keys())
        nse_means = [np.mean(regime_stats[r]['nse']) for r in regimes]
        counts = [regime_stats[r]['count'] for r in regimes]
        
        regime_colors = {'snow_dominated': 'lightblue', 'mixed': 'lightcoral', 'rain_dominated': 'lightgreen'}
        colors = [regime_colors.get(r, 'gray') for r in regimes]
        
        x_pos = range(len(regimes))
        bars = ax5.bar(x_pos, nse_means, alpha=0.7, color=colors)
        ax5.set_xticks(x_pos)
        ax5.set_xticklabels([r.replace('_', '-').title() for r in regimes], rotation=45, ha='right')
        ax5.set_ylabel('Mean NSE')
        ax5.set_title('Performance by Flow Regime')
        ax5.grid(True, alpha=0.3, axis='y')
        
        # Add count labels
        for bar, count in zip(bars, counts):
            ax5.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                    f'n={count}', ha='center', va='bottom', fontsize=8)
    
    # Bias distribution (middle right)
    ax6 = axes[1, 2]
    
    biases = [site['bias'] for site in common_sites]
    ax6.hist(biases, bins=15, color='orange', alpha=0.7, edgecolor='black')
    ax6.axvline(x=0, color='red', linestyle='--', label='Zero bias')
    ax6.set_xlabel('Bias (m³/s)')
    ax6.set_ylabel('Number of Watersheds')
    ax6.set_title('Global Distribution of Streamflow Bias')
    ax6.legend()
    ax6.grid(True, alpha=0.3, axis='y')
    
    # Performance vs watershed area (bottom left)
    ax7 = axes[2, 0]
    
    areas = [site['area_km2'] for site in common_sites if not np.isnan(site['area_km2'])]
    nses = [site['nse'] for site in common_sites if not np.isnan(site['area_km2'])]
    
    if areas and nses:
        scatter7 = ax7.scatter(areas, nses, alpha=0.7, s=40, c='green')
        ax7.set_xlabel('Watershed Area (km²)')
        ax7.set_ylabel('Nash-Sutcliffe Efficiency')
        ax7.set_title('Performance vs Watershed Size')
        ax7.grid(True, alpha=0.3)
        ax7.set_xscale('log')
        ax7.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        ax7.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='NSE = 0.5')
        ax7.legend()
    
    # KGE vs NSE comparison (bottom middle)
    ax8 = axes[2, 1]
    
    nse_vals = [site['nse'] for site in common_sites]
    kge_vals = [site['kge'] for site in common_sites if site['kge'] != -999]
    
    if len(kge_vals) > 0:
        ax8.scatter(nse_vals[:len(kge_vals)], kge_vals, alpha=0.7, s=40, c='purple')
        ax8.set_xlabel('Nash-Sutcliffe Efficiency')
        ax8.set_ylabel('Kling-Gupta Efficiency')
        ax8.set_title('NSE vs KGE Performance')
        ax8.grid(True, alpha=0.3)
        
        # Add reference lines
        ax8.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        ax8.axvline(x=0, color='red', linestyle='--', alpha=0.5)
        ax8.plot([-1, 1], [-1, 1], 'k--', alpha=0.3, label='1:1 line')
        ax8.legend()
    
    # Global performance summary (bottom right)
    ax9 = axes[2, 2]
    
    # Create performance categories
    perf_categories = {
        'Excellent\\n(NSE > 0.75)': len([s for s in common_sites if s['nse'] > 0.75]),
        'Good\\n(0.5 < NSE ≤ 0.75)': len([s for s in common_sites if 0.5 < s['nse'] <= 0.75]),
        'Satisfactory\\n(0.2 < NSE ≤ 0.5)': len([s for s in common_sites if 0.2 < s['nse'] <= 0.5]),
        'Unsatisfactory\\n(NSE ≤ 0.2)': len([s for s in common_sites if s['nse'] <= 0.2])
    }
    
    categories = list(perf_categories.keys())
    counts = list(perf_categories.values())
    colors = ['darkgreen', 'green', 'yellow', 'red']
    
    bars = ax9.bar(range(len(categories)), counts, color=colors, alpha=0.7, edgecolor='black')
    ax9.set_xticks(range(len(categories)))
    ax9.set_xticklabels(categories, rotation=45, ha='right')
    ax9.set_ylabel('Number of Watersheds')
    ax9.set_title('Global Performance Categories')
    ax9.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, count in zip(bars, counts):
        ax9.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.suptitle('CARAVAN Global Large Sample Streamflow Comparison Analysis', 
                 fontsize=18, fontweight='bold')
    plt.tight_layout()
    
    # Save comparison plot
    comparison_path = experiment_dir / 'plots' / 'global_streamflow_comparison_analysis.png'
    plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Global streamflow comparison analysis saved: {comparison_path}")
    
    return common_sites

# Execute Step 3 Analysis
print(f"\n🔍 Step 3.1: Global Streamflow Domain Discovery and Overview")

# Discover completed domains
completed_domains = discover_completed_global_streamflow_domains()

# Create global domain overview map
if len(completed_domains) > 0:
    total_selected, total_discovered, total_with_results, total_with_routing, total_with_obs, total_complete = create_global_streamflow_domain_overview_map(completed_domains)
else:
    print(f"   ⚠️ No completed domains found for overview map")
    total_selected = len(selected_watersheds) if 'selected_watersheds' in locals() else 0
    total_discovered = total_with_results = total_with_routing = total_with_obs = total_complete = 0

print(f"\n🌊 Step 3.2: Global Streamflow Results Extraction")

# Extract streamflow results from simulations
if len(completed_domains) > 0:
    streamflow_results, streamflow_processing_summary = extract_global_streamflow_results_from_domains(completed_domains)
    
    # Load CARAVAN observations
    caravan_obs, obs_summary = load_caravan_global_observations(completed_domains)
else:
    print(f"   ⚠️ No completed domains available for analysis")
    streamflow_results = []
    caravan_obs = {}
    streamflow_processing_summary = {'domains_with_streamflow': 0}
    obs_summary = {'sites_with_streamflow': 0}

print(f"\n🌊 Step 3.3: Global Streamflow Comparison Analysis")

# Create global streamflow comparison analysis
if streamflow_results and caravan_obs:
    common_sites = create_global_streamflow_comparison_analysis(streamflow_results, caravan_obs)
else:
    print(f"   ⚠️  Insufficient data for global streamflow comparison analysis")
    common_sites = None

# Create final global summary report
print(f"\n📋 Creating Final CARAVAN Global Streamflow Study Summary Report...")

summary_report_path = experiment_dir / 'reports' / 'caravan_global_final_report.txt'
summary_report_path.parent.mkdir(parents=True, exist_ok=True)

with open(summary_report_path, 'w') as f:
    f.write("CARAVAN Global Large Sample Streamflow Study - Final Analysis Report\\n")
    f.write("="*72 + "\\n\\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\\n")
    
    f.write("GLOBAL PROCESSING SUMMARY:\\n")
    f.write(f"  Watersheds selected globally: {total_selected}\\n")
    f.write(f"  Processing initiated: {total_discovered}\\n")
    f.write(f"  Simulation results available: {total_with_results}\\n")
    f.write(f"  Routing outputs available: {total_with_routing}\\n")
    f.write(f"  Observations available: {total_with_obs}\\n")
    f.write(f"  Complete streamflow validation: {total_complete}\\n")
    f.write(f"  Streamflow extractions successful: {streamflow_processing_summary['domains_with_streamflow']}\\n")
    f.write(f"  CARAVAN observations available: {obs_summary['sites_with_streamflow']}\\n")
    
    if common_sites:
        f.write(f"  Sites with sim/obs comparison: {len(common_sites)}\\n\\n")
        
        # Global streamflow performance summary
        nse_values = [site['nse'] for site in common_sites]
        kge_values = [site['kge'] for site in common_sites if site['kge'] != -999]
        bias_values = [site['bias'] for site in common_sites]
        corr_values = [site['correlation'] for site in common_sites]
        
        f.write("GLOBAL STREAMFLOW PERFORMANCE SUMMARY:\\n")
        f.write(f"  Mean NSE: {np.mean(nse_values):.3f} ± {np.std(nse_values):.3f}\\n")
        if kge_values:
            f.write(f"  Mean KGE: {np.mean(kge_values):.3f} ± {np.std(kge_values):.3f}\\n")
        f.write(f"  Mean correlation: {np.mean(corr_values):.3f} ± {np.std(corr_values):.3f}\\n")
        f.write(f"  Mean bias: {np.mean(bias_values):+.2f} ± {np.std(bias_values):.2f} m³/s\\n\\n")
        
        # Continental performance breakdown
        continental_performance = {}
        for site in common_sites:
            continent = site['continent']
            if continent not in continental_performance:
                continental_performance[continent] = {'nse': [], 'kge': []}
            continental_performance[continent]['nse'].append(site['nse'])
            if site['kge'] != -999:
                continental_performance[continent]['kge'].append(site['kge'])
        
        f.write("CONTINENTAL PERFORMANCE BREAKDOWN:\\n")
        for continent, performance in continental_performance.items():
            mean_nse = np.mean(performance['nse'])
            mean_kge = np.mean(performance['kge']) if performance['kge'] else 0
            f.write(f"  {continent}: NSE={mean_nse:.3f}, KGE={mean_kge:.3f} ({len(performance['nse'])} sites)\\n")
        
        # Performance categories
        excellent = len([s for s in common_sites if s['nse'] > 0.75])
        good = len([s for s in common_sites if 0.5 < s['nse'] <= 0.75])
        satisfactory = len([s for s in common_sites if 0.2 < s['nse'] <= 0.5])
        unsatisfactory = len([s for s in common_sites if s['nse'] <= 0.2])
        
        f.write("\\nGLOBAL PERFORMANCE CATEGORIES:\\n")
        f.write(f"  Excellent (NSE > 0.75): {excellent} watersheds\\n")
        f.write(f"  Good (0.5 < NSE ≤ 0.75): {good} watersheds\\n")
        f.write(f"  Satisfactory (0.2 < NSE ≤ 0.5): {satisfactory} watersheds\\n")
        f.write(f"  Unsatisfactory (NSE ≤ 0.2): {unsatisfactory} watersheds\\n\\n")
        
        f.write("BEST PERFORMING GLOBAL WATERSHEDS (by NSE):\\n")
        sorted_sites = sorted(common_sites, key=lambda x: x['nse'], reverse=True)
        for i, site in enumerate(sorted_sites[:10]):
            f.write(f"  {i+1}. {site['watershed_id']} ({site['continent']}): NSE={site['nse']:.3f}, KGE={site['kge']:.3f}, Area={site['area_km2']:.0f} km²\\n")

print(f"✅ Final global summary report saved: {summary_report_path}")

print(f"\\n🎉 Step 3 Complete: CARAVAN Global Streamflow Validation Analysis")
print(f"   📁 Results saved to: {experiment_dir}")
print(f"   🌍 Global scope: {total_complete}/{total_selected} watersheds with complete validation")

if common_sites:
    nse_values = [site['nse'] for site in common_sites]
    kge_values = [site['kge'] for site in common_sites if site['kge'] != -999]
    
    print(f"   📊 Global analysis: {len(common_sites)} watersheds with sim/obs comparison")
    print(f"   📈 Global NSE performance: Mean = {np.mean(nse_values):.3f}")
    if kge_values:
        print(f"   📈 Global KGE performance: Mean = {np.mean(kge_values):.3f}")
    
    # Continental summary
    continental_counts = {}
    for site in common_sites:
        continent = site['continent']
        continental_counts[continent] = continental_counts.get(continent, 0) + 1
    
    print(f"   🗺️  Continental validation coverage:")
    for continent, count in continental_counts.items():
        print(f"     {continent}: {count} validated watersheds")
else:
    print(f"   📈 Performance: Awaiting more simulation results for global analysis")

print(f"\\n✅ CARAVAN Global Large Sample Streamflow Analysis Complete!")
print(f"   🌍 Multi-continental streamflow hydrology validation achieved")
print(f"   📊 Statistical patterns identified across global watershed gradients")  
print(f"   🌐 Tutorial series culmination: Regional → Global → Multi-continental analysis!")
print(f"   🏆 Global hydrological understanding through CONFLUENCE framework!")

print(f"\\n🎯 Tutorial Complete: From Point-Scale to Global-Scale Hydrological Modeling")
print(f"   📈 CONFLUENCE Tutorial Series: Energy → Snow → Regional Streamflow → Global Streamflow")
print(f"   🌊 Comprehensive validation across multiple scales and environments")
print(f"   🔬 Scientific advancement from case studies to universal hydrological principles")